In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import json
from sqlalchemy import create_engine, inspect, Table, Column
import pandas as pd
from ilxutils.args_reader import read_args
from ilxutils.scicrunch_client import scicrunch
import sys
import time
from ilxutils.interlex_sql import interlex_sql
import re
from collections import defaultdict
from IPython.core.display import display
import subprocess as sb
import pickle
from pathlib import Path as p
import requests as r
from ilxutils.ttl2pd import *
import difflib

def degrade(var):
    def helper(s):
        return str(re.sub("\(|\)|'|,", "", s).lower().strip())
    if type(var)!=list:
        if var:
            return helper(var)
        else:
            return var
    else:
        return [helper(v) if v else v for v in var]
    
def cj(data, output):
    with open(output+'.json', 'w') as outfile:
        json.dump(data, outfile, indent=4)
    print('Complete')
def oj(filepath):
    with open(filepath+'.json', 'r') as infile:
        return json.load(infile)
def cp(data, output):
    with open(output+'.pickle', 'wb') as outfile:
        pickle.dump(data, outfile)
    print('Complete')
def op(filepath):
    with open(filepath+'.pickle', 'rb') as infile:
        return pickle.load(infile)

In [2]:
args = read_args(api_key= p.home() / 'keys/beta_api_scicrunch_key.txt', db_url= p.home() / 'keys/beta_engine_scicrunch_key.txt', beta=True)
sql = interlex_sql(db_url=args.db_url)
sci = scicrunch(api_key=args.api_key, base_path=args.base_path, db_url=args.db_url)

In [3]:
tables = sql.show_tables()
column_name = list(tables)[0]
tables[tables[column_name].str.contains('dataset')]

,Tables_in_nif_eelg
29,community_dataset_required_fields
30,community_dataset_templates
31,community_datasets
45,dataset_field_annotations
46,dataset_fields
47,dataset_fields_templates
48,dataset_metadata
49,dataset_metadata_fields
50,datasets


In [15]:
df = sql.get_table('dataset_metadata')
df.head()

,id,datasetid,dataset_metadata_id,val


In [3]:
ds = sql.get_table('datasets')
dsf = sql.get_table('dataset_fields')
labs = sql.get_table('labs')
dft = sql.get_table('dataset_fields_templates')

In [5]:
ds.head(1)

,id,uid,name,long_name,description,publications,timestamp,dataset_fields_template_id,lab_status,field_set,record_count
0,3,33352,B&B MASCIS,Beattie Bresnahan MASCIS dataset,Data from 1994-1996 MASCIS study collected at ...,,1501873020,3,pending,Laboratory,NaN


In [6]:
dft.head(1)

,id,uid,timestamp,labid,name,active,parent_id,submitted
0,1,31699,1501006000,1,template 1,0,0,0


In [7]:
dsf.head(1)

,id,dataset_fields_template_id,name,termid,required,position,queryable,multi,multi_suffixes
0,1,1,Laboratory,12818,1,0,1,0,


In [5]:
labs.head()

,id,cid,uid,name,private_description,public_description,timestamp,curated,broadcast_message
0,1,108,31699,Pristionchus lab,Ppa,Ppa,1501005989,approved,None
1,2,97,247,Jeff's Primary Lab,This is my primary lab for testing ODC-SCI,This is Jeff's primary lab for testing ODC-SCI,1501866803,approved,
2,3,97,33353,Adam's Test lab,This adam's test lab,This is Adam's Testlab,1501870636,approved,None
3,4,97,33352,Nielson Lab,Datasets for the Nielson lab,Nielson lab datasets,1501871504,approved,None
4,5,97,33464,Karim Fouad,my lab,Fouad lab,1502472078,approved,None


In [8]:
labs[labs.name.str.contains('Pristionchus lab')]

,id,cid,uid,name,private_description,public_description,timestamp,curated,broadcast_message
0,1,108,31699,Pristionchus lab,Ppa,Ppa,1501005989,approved,None


In [6]:
ds[ds.dataset_fields_template_id == 1]

,id,uid,name,long_name,description,publications,timestamp,dataset_fields_template_id,lab_status,field_set,record_count


In [7]:
dft[dft.labid == 1]

,id,uid,timestamp,labid,name,active,parent_id,submitted
0,1,31699,1501006000,1,template 1,0,0,0
4,5,31699,1502124075,1,template 1,0,1,1
8,9,33572,1502475708,1,template2,0,0,0
9,10,31699,1502485757,1,template 3,0,0,0
13,14,31699,1502984552,1,template 4,0,0,0
19,20,31699,1507574390,1,Template One,1,0,1
32,56,31699,1511897149,1,test 1,1,0,1
40,70,31699,1512774648,1,template-827,0,0,0
42,72,31699,1512776586,1,template-1934,0,0,0
43,73,31699,1512776858,1,template-7aa,0,0,0
